# LLM Router Chain Test
#### *LLM 체인 라우팅 적용하기*

<br><br><hr>

## 00. 기본 설정

In [40]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')
OPENAI_API_KEY = os.getenv('API_KEY_OPENAI')

In [41]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lgdx_team2_routerchain")

LangSmith 추적을 시작합니다.
[프로젝트명]
lgdx_team2_routerchain


In [42]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from operator import itemgetter

# 랭체인 환경 설정
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# VectorDB - FAISS
from langchain_community.vectorstores import FAISS


<br><br><hr>

## 01. 벡터DB 불러오기

In [43]:
### 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

c:\workspaces\LGDXteam2\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [44]:
# 벡터스토어 로드
new_vector_store = FAISS.load_local("../movies_vectorstore_faiss_1500",
                                    embeddings=embeddings,
                                    allow_dangerous_deserialization=True)

<br><br><hr>

## 02. Router Chain: 사용자 질문 유형 구분

- 정보검색
- 추천요청
- 일반대화 (`default_chain`)

In [45]:
# StructuredOutputParser 사용
response_schemas = [
  ResponseSchema(name="type",
                 description="사용자의 입력을 세 가지 범주('정보검색', '추천요청', '일반대화') 중 하나로 구분")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 출력 지시사항 파싱
format_instructions = output_parser.get_format_instructions()

In [46]:
# 1. 사용자 입력값 유형 분류용 프롬프트
classification_template = """사용자의 입력을 다음 세 가지 범주 중 하나로 분류하세요:
1️⃣ **"정보검색"**: 특정 영화, 드라마, 배우, 감독, 러닝타임, 개봉 연도, 수상 내역, 필모그래피 등 **사실적인 정보를 찾는 질문**
   - 기대되는 응답 예시: 배우가 출연한 드라마/영화 목록, 특정 연도의 개봉작 리스트 등
2️⃣ **"추천요청"**: 특정 장르, 배우, 테마(예: 좀비, 시간여행), 감성(예: 힐링, 긴장감) 등에 대한 **추천을 요청하는 질문**
   - 기대되는 응답 예시: 특정 조건을 만족하는 영화/드라마 추천
3️⃣ **"일반대화"**: 서비스와 무관한 일반적인 대화 (예: 날씨, AI 관련 질문, 잡담)

#### **예시 형식**
{format_instructions}

<user_input>
{user_input}
</user_input>
"""
classification_prompt = ChatPromptTemplate.from_template(classification_template,
                                                         partial_variables={'format_instructions': format_instructions})

In [47]:
# 2. LLM을 이용한 질문 유형 분류 체인
classification_chain = (
  classification_prompt
  | ChatGoogleGenerativeAI(model='gemini-1.5-flash', api_key=GEMINI_API_KEY)
  | output_parser
)

In [48]:
# 질문 분류 테스트
print(classification_chain.invoke({'user_input': "2023년에 개봉한 액션 영화 뭐 있어?"}))

# 추천 요청 예상 질문
print(classification_chain.invoke({'user_input': '디카프리오가 주연한 영화 추천해줘.'}))

# 일반 대화 예상 질문
print(classification_chain.invoke({'user_input': "너가 제일 좋아하는 영화 뭐야?"}))

{'type': '추천요청'}
{'type': '추천요청'}
{'type': '일반대화'}


In [49]:
# 딕셔너리 자료형을 string으로 변환하는 함수
def format_change(classification_result: dict, user_input: str) -> str:
    type_value = classification_result.get("classification_result", {}).get("type", "일반대화")
    # keywords = classification_result.get("classification_result", {}).get("keywords", [])

    # string 자료형으로 변경
    # formatted_str = f"type: '{type_value}', keywords: {keywords}, user_input: '{user_input}'"
    formatted_str = f"type: '{type_value}', user_input: '{user_input}'"
    return formatted_str

<br><br><hr>

## 03. Destination Chain

#### *1. default-chain*

In [50]:
# default_chain 생성 (사용자의 의미없는 입력값에 대해 정해진 답변을 할 때)
# 프롬프트 템플릿 설정
default_template = """
"You are a chatbot that must always respond with '🐶: 멍멍!'.
No matter what question the user asks, always reply with '🐶: 멍멍!'"

[사용자 입력]:
{user_input}
[분류 결과]:
{classification_result}
"""
default_prompt = ChatPromptTemplate.from_template(default_template)

# Google Gemini 모델 생성
def load_gemini():
    model = ChatGoogleGenerativeAI(
        model='gemini-1.5-flash',
        temperature=0,
        max_tokens=500,
        api_key=GEMINI_API_KEY
    )
    print(">>>>>>> model loaded...")
    return model

default_llm = load_gemini()

# langchain 체인 구성
default_chain = (
  {"classification_result": RunnablePassthrough(),
   "user_input": RunnablePassthrough()}
  | default_prompt
  | default_llm
  | StrOutputParser()
)

>>>>>>> model loaded...


#### *2. search-chain*

<br><hr>

#### *3. recommendation-chain*
- 추천 목록을 반환하는 체인
- `next_input`이 필요 없음 + 정해진 자료형으로 답해야 함  
  => `RouterOutputParser`를 사용해보는 게 좋을 거 같음..~
- 이 체인 뒤에 사용자 시청 기록 기반으로 반환된 추천 목록에서 5개를 정하는 작업을 해야 함

<br>

- 장르 기반 추천 ⇒ 장르
- 줄거리(키워드/컨셉) 기반 추천 ⇒ 줄거리
- 특정 연도 및 시대별 콘텐츠 추천 ⇒ 줄거리에 언급되는 시대배경/연도
==================================================
- 콘텐츠 정보 기반 추천 ⇒ 어떤 행이 있는지 봐야할듯
- 인기 있는 콘텐츠 추천 ⇒ 시청횟수가 많은 것
- 리뷰(감성) 기반 추천 ⇒ 리뷰
- 사용자 선호 기반 추천 ⇒ 사용자 시청기록


In [78]:
# StructuredOutputParser 사용
recommend_response_schemas = [
  ResponseSchema(name="candidates",
                 description="사용자의 입력에 맞게 추천할 VOD 콘텐츠의 인덱스 리스트. 예: [1, 10, 31, 89, 135, 180]")
]
output_parser = StructuredOutputParser.from_response_schemas(recommend_response_schemas)

# 출력 지시사항 파싱
recommend_chain_format_instructions = output_parser.get_format_instructions()

In [79]:
# 검색기 생성
recommend_chain_retriever = new_vector_store.as_retriever(
    search_type="mmr",   
    search_kwargs={"k": 20,              # 반환할 문서 수 (default: 4)
                   "fetch_k": 50,       # MMR 알고리즘에 전달할 문서 수
                   "lambda_mult": 0.8,  # 결과 다양성 조절 (default: 0.5),
                   }
)

In [80]:
# 프롬프트 템플릿 설정
recommend_chain_template = """
You are a movie-recommendation chatbot.
You must only answer based on the given context.
Do not generate answers that are not directly supported by the context.
사용자의 요청에 따라 추천할 VOD 콘텐츠의 **인덱스 리스트**를 반환하세요.

**중요**:
- 추천 리스트에는 반드시 **[Context]**에서 제공된 문서만 포함해야 합니다.
- **[Context]**에 없는 문서를 절대 생성하거나 포함하지 마세요.
- **[Context]**에 적절한 추천이 없을 경우, 빈 리스트를 반환하세요.

응답은 JSON 형식으로 **오직 추천된 콘텐츠의 인덱스 리스트**만 포함해야 합니다.
{recommend_chain_format_instructions}
---
예제 출력 형식:
```json
{{"candidates": [1, 10, 31, 89, 135, 180]}}```
(만약 적절한 추천이 없을 경우)
```json
{{"candidates": []}}```

[사용자 입력과 사용자 입력값의 유형]:
{formatted_string}

[Context]:
{recommend_chain_retriever}

[Answer]:
"""
recommend_chain_prompt = ChatPromptTemplate.from_template(recommend_chain_template,
                                                          partial_variables={'recommend_chain_format_instructions': recommend_chain_format_instructions})

In [81]:
# LLM 모델 생성 (1. GEMINI 2. OpenAI)
def load_gemini(system_instruction):
    model = ChatGoogleGenerativeAI(
        model='gemini-1.5-flash',
        temperature=0.3,
        max_tokens=5000,
        system_instruction=system_instruction,
        api_key=GEMINI_API_KEY
    )
    print(">>>>>>> Gemini loaded...")
    return model

def load_gpt(system_instruction):
    model = ChatOpenAI(
        model_name='gpt-4o-mini-2024-07-18',
        temperature=0,
        max_tokens=3000,
        api_key=OPENAI_API_KEY
    )
    print(">>>>>>> GPT loaded...")
    return model

system_instruction = """you are a movie-recommendation chatbot. you must answer based on given data."""
recommend_chain_llm = load_gemini(system_instruction)

# langchain 체인 구성
recommend_chain = (
  {"formatted_string":RunnablePassthrough(),
    "recommend_chain_retriever": recommend_chain_retriever,
  }
  | recommend_chain_prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | recommend_chain_llm                  # llm이 원하는 답변을 만듦
  | output_parser
)

>>>>>>> Gemini loaded...


<br><br><hr>

## 04. Full Chain 연결

In [82]:
def process_user_input(classification_result: dict, user_input: str):
    # 사용자의 입력 유형 분류
    print(user_input)
    type_value = classification_result.get("type", "일반대화")  # 기본값 설정
    # keywords = classification_data.get("keywords", [])  # 기본값 설정
    
    print(f"===================== Type: {type_value}")
    # print(f"===================== Keywords: {keywords}")
    
    if type_value == '정보검색':
        return "정보검색 체인 실행은 여기!!!"
    elif type_value == '추천요청':
        formatted_string = format_change(classification_result, user_input)
        candidates = recommend_chain.invoke(formatted_string)
        return candidates
    else:
        return default_chain.invoke({"classification_result": classification_result, "user_input": user_input})

In [83]:
full_chain = (
  {"classification_result": classification_chain,
   "user_input":itemgetter("user_input")}
  | RunnableLambda(lambda x: process_user_input(x["classification_result"], x["user_input"]))
)

In [85]:
full_chain.invoke({"user_input": "액션영화 추천해줘"})

액션영화 추천해줘
===================== Type: 추천요청


{'candidates': [5620, 3309]}

<br><br><hr>

### *데이터 확인*

In [32]:
import pandas as pd

In [33]:
movies = pd.read_csv('../../data/영화_TMDB_5800_Mapping-최종-addIndex.csv', encoding='utf8')

In [34]:
movies[movies['index']==1789]

index          name orgnl_cntry  movie_id  \
1788   1789  돈키호테를 죽인 사나이         스페인    297725   

                                               overview release_date  adult  \
1788  보드카 광고 촬영을 위해 스페인의 작은 마을로 오게 된 잘 나가는 천재 CF 감독 ...   2018-05-19  False   

                         backdrop_path original_language  \
1788  /xr9ZchDO4CwFdJMNoB3I924NuCd.jpg                en   

                      original_title                       poster_path  \
1788  The Man Who Killed Don Quixote  /sjr9cVpq8H7qcmLps8X0Cjn1sxB.jpg   

      popularity  runtime  vote_average  vote_count    genre  test  
1788      11.271      132         6.766        1006  모험, 코미디     1